# Коллаборативная фильтрация

Нужно самостоятельно реализовать колоборативную фильтрацию методами:

1. Knn нужно реализовать 2 базовых метода
    1. Простой KNN (в библиотеке surprise называется KNNBasic)
    2. Непараметрическая регрессия Надарайя-Ватсона (в библиотеке surprise называется KNNWithMeans)
2. SVD-разложение
    1. Метод SGD
    2. Метод ALS
3. SVD++

С полученными методами нужно произвести следующие исследования:
- Нужно сравнить время работы всех реализованных алгоритмов. 
- Нужно сравнить точность (в смысле RMSE) всех реализованных алгоритмов.
- Качество (в смысле RMSE) kNN по параметру k
- Качество (в смысле RMSE) SVD по числу факторов
- Качество (в смысле RMSE) SVD по числу итераций в SGD

В качестве датасэта можно использовать, например, https://grouplens.org/datasets/movielens/ (можно любой другой).

Можно вдохновляться библиотеками (но не копировать код): 
- https://implicit.readthedocs.io/en/latest/quickstart.html 
- https://surprise.readthedocs.io/en/stable/getting_started.html

## Load Data

In [1]:
# ! wget https://files.grouplens.org/datasets/movielens/ml-100k.zip

In [2]:
# ! unzip ml-100k.zip

In [3]:
import pandas as pd
import numpy as np
import os


def load(fname, path='/home/ilya/repos/recsys/hw1/ml-100k'):
    path = os.path.join(path, fname)
    return pd.read_csv(path, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp']).drop(columns=['timestamp'])

df = load('ua.base')
df.describe()

,user_id,item_id,rating
count,90570.000000,90570.000000,90570.000000
mean,461.494038,428.104891,3.523827
std,266.004364,333.088029,1.126073
min,1.000000,1.000000,1.000000
25%,256.000000,174.000000,3.000000
50%,442.000000,324.000000,4.000000
75%,682.000000,636.000000,4.000000
max,943.000000,1682.000000,5.000000


In [4]:
def load_train():
    return load('ua.base')
def load_test():
    return load('ua.test')

df_train = load_train()
df_test = load_test()

Разделение на сплиты немного с подлецой:

In [5]:
print(len(set(df_test.item_id.unique()) - set(df_train.item_id.unique())))
print(len(set(df_test.user_id.unique()) - set(df_train.user_id.unique())))
print(len(set(df_train.item_id.unique()) - set(df_test.item_id.unique())))
print(len(set(df_train.user_id.unique()) - set(df_test.user_id.unique())))

2
0
553
0


## Demo

### KNN

Для каждой пары `(u, i)` найти $k$ ближайших к `u` пользователей таких, что они оценили как минимум `min_support` таких же фильмов, что и `u`. За расшифровкой математических обозначений, использованных ниже, обращайтесь в доки surprise: https://surprise.readthedocs.io/en/stable/notation_standards.html#notation-standards

Реализованы расстояния `cosine`, `msd`, повторяющие поведение сюрпрайза: https://surprise.readthedocs.io/en/stable/similarities.html

Аггрегация оценок ближайших соседей для предсказания соответствует KNNBasic и KNNWithMeans: https://surprise.readthedocs.io/en/stable/knn_inspired.html

In [6]:
metric = 'msd'
min_support = 5
k = 5
with_means = True

In [7]:
from colfil.knn import knn_user_based

df_train_, df_test_ = knn_user_based(df_train, df_test, k, metric, min_support, verbose=True, with_means=False)

...preprocessing
...counting common items
...computing similarities
...finding neighbors
...predicting


In [8]:
imp = df_test_.loc[df_test_.impossible]
print(len(imp))
imp.head()

3


,user_id,item_id,rating,_user_id,_item_id,pred_rating,impossible
2415,242,361,5,241,360,3.523827,True
4049,405,1582,1,404,1581,3.523827,True
6749,675,1653,5,674,1652,3.523827,True


In [9]:
def rmse_knn(df):
    df_possible = df[df.impossible == False]
    return ((df_possible.rating - df_possible.pred_rating) ** 2).mean() ** 0.5

rmse_knn(df_test_)

1.1535689380152743

### SVD SGD

Реализован алгоритм, описанный в доках сюрпрайза: https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD

In [10]:
hparams = dict(
    n_factors=100,
    n_epochs=10,
    batch_size=128,
    init_mean=0,
    init_std_dev=.1,
    biased=False,
    lr=.005,
    reg=.02,
    random_state=None,
    return_logs=False
)

In [11]:
from colfil import svd

df_train_, df_test_ = svd(df_train, df_test, **hparams)

In [12]:
def rmse(df):
    return ((df.rating - df.pred_rating) ** 2).mean() ** 0.5

rmse(df_test_)

1.059415294295266

### SVD ALS

Введем обозначения:
- $r_u\in\mathbb{R}^{n}$ --- все оценки пользователя $u$
- $p_u\in\mathbb{R}^{f}$ --- факторы (эмбеддинг) пользователя $u$
- $Q\in\mathbb{R}^{n\times f}$ --- факторы всех предметов, которые оценил $u$

Ошибка: $f(p_u) = {1\over2}\|r_u-Qp_u\|_2^2+{1\over2}\lambda\|p_u\|_2^2$. Выведем градиент:
$$
\begin{align*}
df&={1\over2}d\langle r_u-Qp_u,r_u-Qp_u\rangle+{1\over2}\lambda d\|p_u\|_2^2\\
&=\langle r_u-Qp_u, r_u-Qdp_u\rangle+\lambda \langle p_u,dp_u\rangle\\
&=-(r_u-Qp_u)^TQdp_u+\lambda p_u^Tdp_u\\
&=\langle-Q^T(r_u-Qp_u)+\lambda p_u,dp_u\rangle\\
\Rightarrow \nabla_{p_u}f&=-Q^T(r_u-Qp_u)+\lambda p_u
\end{align*}
$$

Точка минимума задается уравнением $(Q^TQ+\lambda I_f)p_u=Q^Tr_u$. Случай с предметами $i$, а не пользователями, аналогичен.


In [13]:
hparams = dict(
    n_factors=100,
    n_epochs=10,
    init_mean=0,
    init_std_dev=.1,
    # biased=False,
    reg=.02,
    random_state=None,
    verbose=True
)

In [14]:
from colfil import svdals

df_train_, df_test_ = svdals(df_train, df_test, **hparams)

i_epoch=0 train_rmse=0.5970736904185785
i_epoch=1 train_rmse=0.24188672348574353
i_epoch=2 train_rmse=0.15943656798237774
i_epoch=3 train_rmse=0.12019551610714738
i_epoch=4 train_rmse=0.0969566079947624
i_epoch=5 train_rmse=0.08155087790723797
i_epoch=6 train_rmse=0.07048416599394934
i_epoch=7 train_rmse=0.06202613146342693
i_epoch=8 train_rmse=0.05531394783221021
i_epoch=9 train_rmse=0.049863771104949176


In [15]:
def rmse(df):
    return ((df.rating - df.pred_rating) ** 2).mean() ** 0.5

rmse(df_test_)

1.8362499288284264

### SVD++

Реализован алгоритм сюрпрайза: https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVDpp

In [16]:
hparams = dict(
    n_factors=100,
    n_epochs=5,
    batch_size=128,
    init_mean=0,
    init_std_dev=.1,
    biased=False,
    lr=.005,
    reg=.02,
    random_state=None,
    return_logs=False,
    verbose=True
)

In [17]:
from colfil import svdpp

df_train_, df_test_ = svdpp(df_train, df_test, **hparams)

=== i_epoch=0 ===
i_batch=0, rmse=1.0597
i_batch=177, rmse=1.1689
i_batch=354, rmse=1.1224
i_batch=531, rmse=1.0693

=== i_epoch=1 ===
i_batch=0, rmse=1.0519
i_batch=177, rmse=1.0005
i_batch=354, rmse=0.9660
i_batch=531, rmse=1.0240

=== i_epoch=2 ===
i_batch=0, rmse=1.0168
i_batch=177, rmse=1.0458
i_batch=354, rmse=0.9937
i_batch=531, rmse=0.9493

=== i_epoch=3 ===
i_batch=0, rmse=1.0334
i_batch=177, rmse=1.0444
i_batch=354, rmse=0.9040
i_batch=531, rmse=1.0238

=== i_epoch=4 ===
i_batch=0, rmse=0.9368
i_batch=177, rmse=0.8506
i_batch=354, rmse=0.9528
i_batch=531, rmse=0.9196



In [18]:
def rmse(df):
    return ((df.rating - df.pred_rating) ** 2).mean() ** 0.5

rmse(df_test_)

1.1073097835054937

## Experiments

In [19]:
from time import time
import itertools as it
from collections import defaultdict


def my_grid_search(param_grid, const_parameters, func, rmse_callable, verbose=False):
    res = defaultdict(list)
    keys = param_grid.keys()
    base_vals = param_grid.values()
    for cur_params in it.product(*base_vals):
        args = {key: val for key, val in zip(keys, cur_params)}
        if verbose:
            print(args)
        
        start = time()
        _, df_test_ = func(**args, **const_parameters)
        end = time()

        res['time'].append(end - start)
        res['rmse'].append(rmse_callable(df_test_))
        
        for name, val in args.items():
            res[name].append(val)
    return res
    

### KNN

In [20]:
from colfil import knn_user_based


param_grid = dict(
    metric = ['msd', 'cosine'],
    min_support = range(1, 50),
    k = range(1, 50),
    with_means = [False, True]
)

def rmse_knn(df):
    # df = df[df.impossible == False]
    return ((df.rating - df.pred_rating) ** 2).mean() ** 0.5

res = my_grid_search(
    param_grid,
    const_parameters=dict(df_train=df_train, df_test=df_test),
    func=knn_user_based,
    rmse_callable=rmse_knn,
    verbose=True
)

{'metric': 'msd', 'min_support': 1, 'k': 1, 'with_means': False}
{'metric': 'msd', 'min_support': 1, 'k': 1, 'with_means': True}
{'metric': 'msd', 'min_support': 1, 'k': 2, 'with_means': False}
{'metric': 'msd', 'min_support': 1, 'k': 2, 'with_means': True}
{'metric': 'msd', 'min_support': 1, 'k': 3, 'with_means': False}


In [ ]:
import dill

dill.dump_session('session.db')

### SVD SGD

In [ ]:
from colfil import svd
import numpy as np

param_grid = dict(
    n_factors = range(64, 513, 64),
    lr=np.logspace(-3, -1, 10),
    reg=np.logspace(-3, -1, 10),
)

def rmse(df):
    return ((df.rating - df.pred_rating) ** 2).mean() ** 0.5

res_1 = my_grid_search(
    param_grid,
    const_parameters=dict(df_train=df_train, df_test=df_test),
    func=svd,
    rmse_callable=rmse,
    verbose=True
)

In [ ]:
import dill

dill.dump_session('session_1.db')

### SVD ALS

In [ ]:
from colfil import svdals
import numpy as np

param_grid = dict(
    n_factors = range(64, 513, 64),
    reg=np.logspace(-3, -1, 10),
)

def rmse(df):
    return ((df.rating - df.pred_rating) ** 2).mean() ** 0.5

res_2 = my_grid_search(
    param_grid,
    const_parameters=dict(df_train=df_train, df_test=df_test),
    func=svdals,
    rmse_callable=rmse,
    verbose=True
)

In [ ]:
import dill

dill.dump_session('session_2.db')

### SVD++

In [ ]:
from colfil import svdpp
import numpy as np

param_grid = dict(
    n_factors = range(64, 513, 64),
    lr=np.logspace(-3, -1, 10),
    reg=np.logspace(-3, -1, 10),
)

def rmse(df):
    return ((df.rating - df.pred_rating) ** 2).mean() ** 0.5

res_3 = my_grid_search(
    param_grid,
    const_parameters=dict(df_train=df_train, df_test=df_test),
    func=svdpp,
    rmse_callable=rmse,
    verbose=True
)

In [ ]:
import dill

dill.dump_session('session_3.db')